# Titanic Bayes Exercise - Katherine Olsen

In [1]:
import pandas as pd
import numpy as np
import sklearn 

from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

C:\Users\KathyOlsen\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
filename = "titanic3.xls"
df = pd.read_excel(filename)

df.head() 

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [3]:
df1=df
df1=df1.drop('boat',axis=1)
df1=df1.drop('body',axis=1)
df1=df1.drop('home.dest',axis=1)
df1=df1.drop('fare',axis=1)
df1=df1.drop('ticket',axis=1)
df1=df1.drop('cabin',axis=1)

In [4]:
df1['fam']=df1['sibsp']+df1['parch']
df1.head()

,pclass,survived,name,sex,age,sibsp,parch,embarked,fam
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,S,0
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,S,3
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,S,3
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,S,3
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,S,3


In [5]:
df1.count()

pclass      1309
survived    1309
name        1309
sex         1309
age         1046
sibsp       1309
parch       1309
embarked    1307
fam         1309
dtype: int64

In [6]:
df2=df1
df2=df2.dropna(axis=0,subset=['age'])
df2=df2.dropna(axis=0,subset=['embarked'])
df2.describe()

,pclass,survived,age,sibsp,parch,fam
count,1044.000000,1044.000000,1044.000000,1044.000000,1044.000000,1044.000000
mean,2.209770,0.407088,29.842593,0.503831,0.421456,0.925287
std,0.840639,0.491527,14.390746,0.912775,0.840353,1.453720
min,1.000000,0.000000,0.166700,0.000000,0.000000,0.000000
25%,1.000000,0.000000,21.000000,0.000000,0.000000,0.000000
50%,2.000000,0.000000,28.000000,0.000000,0.000000,0.000000
75%,3.000000,1.000000,39.000000,1.000000,1.000000,1.000000
max,3.000000,1.000000,80.000000,8.000000,6.000000,10.000000


In [7]:
df2.count()

pclass      1044
survived    1044
name        1044
sex         1044
age         1044
sibsp       1044
parch       1044
embarked    1044
fam         1044
dtype: int64

In [8]:
def withfam(x):
    if x>=1: return 1
    else: return 0
    
df2['withfam']=df2['fam'].apply(withfam)

def nofam(x):
    if x==0: return 1
    else: return 0
    
df2['nofam']=df2['fam'].apply(nofam)

def pclass1(x):
    if x==1: return 1
    else: return 0
    
df2['pclass1']=df2['pclass'].apply(pclass1)

def pclass2(x):
    if x==2: return 1
    else: return 0
    
df2['pclass2']=df2['pclass'].apply(pclass2) 

def pclass3(x):
    if x==3: return 1
    else: return 0
    
df2['pclass3']=df2['pclass'].apply(pclass3) 

def female(x):
    if x=='female': return 1
    else: return 0
    
df2['female']=df2['sex'].apply(female) 

def male(x):
    if x=='male': return 1
    else: return 0
    
df2['male']=df2['sex'].apply(male)

def adult(x):
    if x>=18: return 1
    else: return 0
    
df2['adult']=df2['age'].apply(adult)

def child2andup(x):
    if x<18 and x>=2: return 1
    else: return 0
    
df2['child2andup']=df2['age'].apply(child2andup)

def childunder2(x):
    if x<2: return 1
    else: return 0
    
df2['childunder2']=df2['age'].apply(childunder2)

def England(x):
    if x=='S': return 1
    else: return 0
    
df2['England']=df2['embarked'].apply(England)

def France(x):
    if x=='C': return 1
    else: return 0
    
df2['France']=df2['embarked'].apply(France)

def Ireland(x):
    if x=='Q': return 1
    else: return 0
    
df2['Ireland']=df2['embarked'].apply(Ireland)

In [9]:
df2.head(10)

,pclass,survived,name,sex,age,sibsp,parch,embarked,fam,withfam,...,pclass2,pclass3,female,male,adult,child2andup,childunder2,England,France,Ireland
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,S,0,0,...,0,0,1,0,1,0,0,1,0,0
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,S,3,1,...,0,0,0,1,0,0,1,1,0,0
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,S,3,1,...,0,0,1,0,0,1,0,1,0,0
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,S,3,1,...,0,0,0,1,1,0,0,1,0,0
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,S,3,1,...,0,0,1,0,1,0,0,1,0,0
5,1,1,"Anderson, Mr. Harry",male,48.0000,0,0,S,0,0,...,0,0,0,1,1,0,0,1,0,0
6,1,1,"Andrews, Miss. Kornelia Theodosia",female,63.0000,1,0,S,1,1,...,0,0,1,0,1,0,0,1,0,0
7,1,0,"Andrews, Mr. Thomas Jr",male,39.0000,0,0,S,0,0,...,0,0,0,1,1,0,0,1,0,0
8,1,1,"Appleton, Mrs. Edward Dale (Charlotte Lamson)",female,53.0000,2,0,S,2,1,...,0,0,1,0,1,0,0,1,0,0
9,1,0,"Artagaveytia, Mr. Ramon",male,71.0000,0,0,C,0,0,...,0,0,0,1,1,0,0,0,1,0


In [10]:
df3=df2
df3=df3.drop('pclass',axis=1)
df3=df3.drop('sex',axis=1)
df3=df3.drop('name',axis=1)
df3=df3.drop('embarked',axis=1)
df3.head()

,survived,age,sibsp,parch,fam,withfam,nofam,pclass1,pclass2,pclass3,female,male,adult,child2andup,childunder2,England,France,Ireland
0,1,29.0000,0,0,0,0,1,1,0,0,1,0,1,0,0,1,0,0
1,1,0.9167,1,2,3,1,0,1,0,0,0,1,0,0,1,1,0,0
2,0,2.0000,1,2,3,1,0,1,0,0,1,0,0,1,0,1,0,0
3,0,30.0000,1,2,3,1,0,1,0,0,0,1,1,0,0,1,0,0
4,0,25.0000,1,2,3,1,0,1,0,0,1,0,1,0,0,1,0,0


In [11]:
df3.corr()

,survived,age,sibsp,parch,fam,withfam,nofam,pclass1,pclass2,pclass3,female,male,adult,child2andup,childunder2,England,France,Ireland
survived,1.000000,-0.058965,-0.010959,0.115783,0.060049,0.209782,-0.209782,0.281904,0.039397,-0.284691,0.536613,-0.536613,-0.100660,0.060216,0.109194,-0.172422,0.221474,-0.067148
age,-0.058965,1.000000,-0.242910,-0.149993,-0.239227,-0.126818,0.126818,0.390809,-0.013482,-0.335649,-0.067410,0.067410,0.599814,-0.512099,-0.295979,-0.071752,0.087368,-0.018907
sibsp,-0.010959,-0.242910,1.000000,0.374126,0.844161,0.627098,-0.627098,-0.016742,-0.054554,0.062163,0.096829,-0.096829,-0.403864,0.383809,0.109031,0.041175,-0.046487,0.003975
parch,0.115783,-0.149993,0.374126,1.000000,0.812980,0.569777,-0.569777,-0.004753,-0.021069,0.022485,0.222810,-0.222810,-0.315458,0.258612,0.180456,0.022150,0.001846,-0.048452
fam,0.060049,-0.239227,0.844161,0.812980,1.000000,0.723120,-0.723120,-0.013260,-0.046433,0.052029,0.189597,-0.189597,-0.435939,0.390486,0.172776,0.038658,-0.028121,-0.025513
withfam,0.209782,-0.126818,0.627098,0.569777,0.723120,1.000000,-1.000000,0.142780,0.022300,-0.146224,0.273649,-0.273649,-0.298088,0.246062,0.166607,-0.055960,0.112346,-0.098027
nofam,-0.209782,0.126818,-0.627098,-0.569777,-0.723120,-1.000000,1.000000,-0.142780,-0.022300,0.146224,-0.273649,0.273649,0.298088,-0.246062,-0.166607,0.055960,-0.112346,0.098027
pclass1,0.281904,0.390809,-0.016742,-0.004753,-0.013260,0.142780,-0.142780,1.000000,-0.351226,-0.584341,0.119476,-0.119476,0.161803,-0.140565,-0.074235,-0.299679,0.379321,-0.106132
pclass2,0.039397,-0.013482,-0.054554,-0.021069,-0.046433,0.022300,-0.022300,-0.351226,1.000000,-0.554572,0.029786,-0.029786,0.034309,-0.046592,0.023102,0.186225,-0.159466,-0.077693
pclass3,-0.284691,-0.335649,0.062163,0.022485,0.052029,-0.146224,0.146224,-0.584341,-0.554572,1.000000,-0.132002,0.132002,-0.173540,0.165310,0.045954,0.104936,-0.198911,0.161665


In [12]:
# The strongest correlations are female/male (.537/-.537), pclass3 (-.285), pclass1 (.282),
# France (.221), withfam/nofam (.210/-.210), and England (-.172).  The other correlations are .116 or lower.
# Since withfam/nofam has stronger correlations than sibsp and parch, I will drop the sibsp, parch, and fam
# columns.  In addition, since the adult, child2andup, and childunder2 columns each have higher correlations
# to survival than age, I will drop the age column.

df3=df3.drop('sibsp',axis=1)
df3=df3.drop('parch',axis=1)
df3=df3.drop('fam',axis=1)
df3=df3.drop('age',axis=1)
df3.head()

,survived,withfam,nofam,pclass1,pclass2,pclass3,female,male,adult,child2andup,childunder2,England,France,Ireland
0,1,0,1,1,0,0,1,0,1,0,0,1,0,0
1,1,1,0,1,0,0,0,1,0,0,1,1,0,0
2,0,1,0,1,0,0,1,0,0,1,0,1,0,0
3,0,1,0,1,0,0,0,1,1,0,0,1,0,0
4,0,1,0,1,0,0,1,0,1,0,0,1,0,0


In [13]:
df3.dtypes

survived       int64
withfam        int64
nofam          int64
pclass1        int64
pclass2        int64
pclass3        int64
female         int64
male           int64
adult          int64
child2andup    int64
childunder2    int64
England        int64
France         int64
Ireland        int64
dtype: object

# Predicting Survival Using Various Models

# 1. Naive Bayes - Gaussian

In [14]:
modeldf=df3
X = modeldf.drop('survived',axis=1)
y = modeldf['survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=101)
gnb = GaussianNB()
gnb.fit(X_train, y_train)

GaussianNB(priors=None)

In [15]:
gnb.score(X_train, y_train)

0.7739463601532567

In [16]:
y_pred = gnb.predict(X_test)

cm = pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    columns=['Predicted Did Not Survive', 'Predicted Survived'],
    index=['True Did Not Survive', 'True Survived']
)

cm

,Predicted Did Not Survive,Predicted Survived
True Did Not Survive,139,19
True Survived,35,68


In [17]:
y_test.value_counts()

0    158
1    103
Name: survived, dtype: int64

In [18]:
gnb.score(X_test, y_test)

0.7931034482758621

In [19]:
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.80      0.88      0.84       158
          1       0.78      0.66      0.72       103

avg / total       0.79      0.79      0.79       261



# 2. Naive Bayes - Bernoulli

In [20]:
bnb = BernoulliNB()
bnb.fit(X_train, y_train)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [21]:
bnb.score(X_train, y_train)

0.7752234993614304

In [22]:
y_pred = bnb.predict(X_test)

cm = pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    columns=['Predicted Did Not Survive', 'Predicted Survived'],
    index=['True Did Not Survive', 'True Survived']
)

cm

,Predicted Did Not Survive,Predicted Survived
True Did Not Survive,143,15
True Survived,34,69


In [23]:
bnb.score(X_test, y_test)

0.8122605363984674

In [24]:
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.81      0.91      0.85       158
          1       0.82      0.67      0.74       103

avg / total       0.81      0.81      0.81       261



# 3. Logistic Regression

In [25]:
LogReg = LogisticRegression()
LogReg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [26]:
LogReg.score(X_train, y_train)

0.789272030651341

In [27]:
y_pred = LogReg.predict(X_test)

cm = pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    columns=['Predicted Did Not Survive', 'Predicted Survived'],
    index=['True Did Not Survive', 'True Survived']
)

cm

,Predicted Did Not Survive,Predicted Survived
True Did Not Survive,145,13
True Survived,36,67


In [28]:
LogReg.score(X_test, y_test)

0.8122605363984674

In [29]:
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.80      0.92      0.86       158
          1       0.84      0.65      0.73       103

avg / total       0.82      0.81      0.81       261



# 4. Decision Tree

In [30]:
model = tree.DecisionTreeClassifier()
model.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [31]:
accuracy_score(y_test, y_pred)

0.8122605363984674

In [32]:
y_predict = model.predict(X_test)

cm = pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    columns=['Predicted Did Not Survive', 'Predicted Survived'],
    index=['True Did Not Survive', 'True Survived']
)

cm

,Predicted Did Not Survive,Predicted Survived
True Did Not Survive,145,13
True Survived,36,67


In [33]:
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.80      0.92      0.86       158
          1       0.84      0.65      0.73       103

avg / total       0.82      0.81      0.81       261



# Summary

In [34]:
print("Modeling Type - Avg/Total Precision")
print("Bayes-Gaussian - 0.79")
print("Bayes-Bernoulli - 0.81")
print("Logistic Regression - 0.82")
print("Decision Tree - 0.82")
print(" ")
print("In this case Logistic Regression and Decision Tree yielded the same and best results.")

Modeling Type - Avg/Total Precision
Bayes-Gaussian - 0.79
Bayes-Bernoulli - 0.81
Logistic Regression - 0.82
Decision Tree - 0.82
 
In this case Logistic Regression and Decision Tree yielded the same and best results.


# Comparison: Dropping Age-Related Variables and Keeping Rows with Missing Age Values

In [35]:
# For curiousity's sake . . .
# What if we dropped the Age (and other age-related) columns in favor of keeping more rows of data  . . .
df4=df1
df4=df4.dropna(axis=0,subset=['embarked'])
df4=df4.drop('age',axis=1)
df4['withfam']=df4['fam'].apply(withfam)
df4['nofam']=df4['fam'].apply(nofam)
df4['pclass1']=df4['pclass'].apply(pclass1)
df4['pclass2']=df4['pclass'].apply(pclass2) 
df4['pclass3']=df4['pclass'].apply(pclass3) 
df4['female']=df4['sex'].apply(female) 
df4['male']=df4['sex'].apply(male)
df4['England']=df4['embarked'].apply(England)
df4['France']=df4['embarked'].apply(France) 
df4['Ireland']=df4['embarked'].apply(Ireland)

df4.head()

,pclass,survived,name,sex,sibsp,parch,embarked,fam,withfam,nofam,pclass1,pclass2,pclass3,female,male,England,France,Ireland
0,1,1,"Allen, Miss. Elisabeth Walton",female,0,0,S,0,0,1,1,0,0,1,0,1,0,0
1,1,1,"Allison, Master. Hudson Trevor",male,1,2,S,3,1,0,1,0,0,0,1,1,0,0
2,1,0,"Allison, Miss. Helen Loraine",female,1,2,S,3,1,0,1,0,0,1,0,1,0,0
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,1,2,S,3,1,0,1,0,0,0,1,1,0,0
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,1,2,S,3,1,0,1,0,0,1,0,1,0,0


In [36]:
df5=df4
df5=df5.drop('pclass',axis=1)
df5=df5.drop('sex',axis=1)
df5=df5.drop('name',axis=1)
df5=df5.drop('embarked',axis=1)
df5.head()

,survived,sibsp,parch,fam,withfam,nofam,pclass1,pclass2,pclass3,female,male,England,France,Ireland
0,1,0,0,0,0,1,1,0,0,1,0,1,0,0
1,1,1,2,3,1,0,1,0,0,0,1,1,0,0
2,0,1,2,3,1,0,1,0,0,1,0,1,0,0
3,0,1,2,3,1,0,1,0,0,0,1,1,0,0
4,0,1,2,3,1,0,1,0,0,1,0,1,0,0


In [37]:
df5.count()

survived    1307
sibsp       1307
parch       1307
fam         1307
withfam     1307
nofam       1307
pclass1     1307
pclass2     1307
pclass3     1307
female      1307
male        1307
England     1307
France      1307
Ireland     1307
dtype: int64

In [38]:
df5.corr()

,survived,sibsp,parch,fam,withfam,nofam,pclass1,pclass2,pclass3,female,male,England,France,Ireland
survived,1.000000,-0.026931,0.083642,0.028005,0.203651,-0.203651,0.277038,0.051873,-0.281916,0.527458,-0.527458,-0.152055,0.183379,-0.015465
sibsp,-0.026931,1.000000,0.373383,0.861899,0.590884,-0.590884,-0.033059,-0.052818,0.071891,0.110768,-0.110768,0.074227,-0.048788,-0.048926
parch,0.083642,0.373383,1.000000,0.792225,0.548829,-0.548829,-0.011873,-0.010413,0.018800,0.214371,-0.214371,0.072362,-0.008985,-0.101186
fam,0.028005,0.861899,0.792225,1.000000,0.688682,-0.688682,-0.028236,-0.040436,0.057567,0.190041,-0.190041,0.088381,-0.037004,-0.087493
withfam,0.203651,0.590884,0.548829,0.688682,1.000000,-1.000000,0.129084,0.034457,-0.139795,0.286747,-0.286747,-0.013444,0.107317,-0.127688
nofam,-0.203651,-0.590884,-0.548829,-0.688682,-1.000000,1.000000,-0.129084,-0.034457,0.139795,-0.286747,0.286747,0.013444,-0.107317,0.127688
pclass1,0.277038,-0.033059,-0.011873,-0.028236,0.129084,-0.129084,1.000000,-0.295893,-0.621279,0.104162,-0.104162,-0.184042,0.327916,-0.165641
pclass2,0.051873,-0.052818,-0.010413,-0.040436,0.034457,-0.034457,-0.295893,1.000000,-0.564669,0.029976,-0.029976,0.197157,-0.135134,-0.122263
pclass3,-0.281916,0.071891,0.018800,0.057567,-0.139795,0.139795,-0.621279,-0.564669,1.000000,-0.114587,0.114587,-0.002719,-0.172468,0.243410
female,0.527458,0.110768,0.214371,0.190041,0.286747,-0.286747,0.104162,0.029976,-0.114587,1.000000,-1.000000,-0.116741,0.067720,0.089445


In [39]:
df6=df5
df6=df6.drop('sibsp',axis=1)
df6=df6.drop('parch',axis=1)
df6=df6.drop('fam',axis=1)
df6.head()

,survived,withfam,nofam,pclass1,pclass2,pclass3,female,male,England,France,Ireland
0,1,0,1,1,0,0,1,0,1,0,0
1,1,1,0,1,0,0,0,1,1,0,0
2,0,1,0,1,0,0,1,0,1,0,0
3,0,1,0,1,0,0,0,1,1,0,0
4,0,1,0,1,0,0,1,0,1,0,0


In [40]:
modeldfalt=df6
X = modeldfalt.drop('survived',axis=1)
y = modeldfalt['survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=101)
gnb = GaussianNB()
gnb.fit(X_train, y_train)

GaussianNB(priors=None)

In [41]:
gnb.score(X_train, y_train)

0.7663265306122449

In [42]:
y_pred = gnb.predict(X_test)

cm = pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    columns=['Predicted Did Not Survive', 'Predicted Survived'],
    index=['True Did Not Survive', 'True Survived']
)

cm

,Predicted Did Not Survive,Predicted Survived
True Did Not Survive,155,50
True Survived,28,94


In [43]:
y_test.value_counts()

0    205
1    122
Name: survived, dtype: int64

In [44]:
gnb.score(X_test, y_test)

0.7614678899082569

In [45]:
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.85      0.76      0.80       205
          1       0.65      0.77      0.71       122

avg / total       0.77      0.76      0.76       327



In [46]:
# Although the precision for Did Not Survive (0) is higher than in the first model, the precision for 
# Survived (1) is significantly lower and the avg/total Precision is lower than in the first model.
# Accordingly, removing age-related variables lowers the predictive ability of the Bayes-Gaussian model.